# Genie Top200 Chart

In [1]:
import requests
import pandas as pd 


### 1. 인터넷에서 데이터 가져오기

In [3]:
# Genie top200
url = 'https://www.genie.co.kr/chart/top200'
req = requests.get(url)         #  기본 html로 받을 것이므로 .jason 등 해줄 필요 x 
html = req.text                 # 이거 뭐? 
html 

# 잘못 가져온 거라 함 

'<br>\r\n<br>\r\n<center>\r\n<img src="http://www.geniemusic.co.kr/images/common/logo_r1.png"><br>\r\n<h2> <meta http-equiv="Content-Type" content="text/html;charset=UTF-8"> ì\xa0\x91ì\x86\x8dì\x9a\x94ì²\xadì\x9d´ ë³´ì\x95\x88ì\xa0\x95ì±\x85ì\x97\x90 ì\x9d\x98í\x95´ ì°¨ë\x8b¨ë\x90\x98ì\x97\x88ì\x8aµë\x8b\x88ë\x8b¤. ë\x8b¹ì\x82¬ ê³\xa0ê°\x9dì\x84¼í\x84°ë¡\x9c ë¬¸ì\x9d\x98í\x95´ì£¼ì\x8b\xadì\x8b\x9cì\x98¤.<br><br>\r\nThe security policy of the connection request is blocked. Contact your customer service representative.<br><br>\r\nì§\x80ë\x8b\x88ë®¤ì§\x81 ê³\xa0ê°\x9dì\x84¼í\x84° 1577-5337<br><br>\r\n</h2>\r\n</center>\r\n<br>'

In [5]:
# Chrome User-Agent
header = {'User-Agent': 
          "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.122 Safari/537.36"
} 
req = requests.get(url, headers = header)
html = req.text
#html

# html 출력시키지 X 깃 허브 문제생김 (용량? )

In [9]:
from bs4 import BeautifulSoup

soup= BeautifulSoup(html, 'html.parser')

### 찾으려고 하는 데이터의 노드 찾기

- top 50 음악리스트만 찾고싶어

In [10]:
trs = soup.find_all('tr')
len(trs)

51

In [12]:
# top 50은 50뿐이어야 하는ㄷ ㅔ
# 제대로 50만 찾기

In [18]:
tbody = soup.select_one('table.list-wrap').find('tbody')
trs = tbody.select('tr')
len(trs)

50

In [17]:
# 이거 놓쳤어(tbody에서 우클릭 - copy selector)
# 개발자 도구에서 찾아주는 Selector 
tbody = soup.select_one('#body-content > div.newest-ist > div > table > tbody')
trs = tbody.select('tr')
len(trs)

AttributeError: 'NoneType' object has no attribute 'select'

In [ ]:
#body-content > div.newest-list > div > table > tbody

- Xpath 값은 아래와 같으나 BeautifulSoup은 지원하지 않음
<br>
//*[@id="body-content"]/div[6]/div/table/tbody

### 3. 여러개의 데이터 중 하나를 선택해서 원하는 정보를 추츨
- rank, title, artist, album

- 지니차트 F12랑 병행하며 보세요

In [21]:
tr = trs[0]
# 길어서 출력X

In [23]:
# rank 
tr.select_one('.number').get_text()

'1\n                                        \n                                    \n유지\n\n'

In [29]:
# 스트링 전체를 split해준 다음, 인덱스로 원하는 rank 숫자만 꺼내기
rank = tr.select_one('.number').get_text().split('\n')[0]
rank

'1'

In [30]:
# title
# td class info 의 타이틀 빼오면 될 것 같음 

In [31]:
tr.select_one('.title.ellipsis').get_text()        # 스페이스 남겨두면 X 

'\n                                        \n                                            \n                                        \n                                        \n                                        \n                                        \n                                            \n                                                리무진 (Feat. MINO) (Prod. by GRAY)'

In [32]:
title = tr.select_one('.title.ellipsis').get_text().strip()
title

'리무진 (Feat. MINO) (Prod. by GRAY)'

In [35]:
# artist
artist = tr.select_one('.artist.ellipsis').get_text().strip()
artist

"BE'O (비오)"

In [36]:
# album
album = tr.select_one('.albumtitle.ellipsis').get_text().strip()
album

'쇼미더머니 10 Episode 3'

### 4. 한 페이지에 있는 모든 데이터를 반복문으로 가져와 데프 만들기

- 개별 리스트 만들기 ( rank_list, title_list...)
- '순위' : rank_list, ... 
- or 
- 딕셔너리의 리스트 만들기 
- {}

1. 반복문으로 리스트 생성 - 리스트로 데프 생성

In [37]:
rank_list, title_list, artist_list, album_list = [],[],[],[]      # 반복문 전에 빈 리스트 만들기
for tr in trs:
    rank = tr.select_one('.number').get_text().split('\n')[0]
    title = tr.select_one('.title.ellipsis').get_text().strip()
    artist = tr.select_one('.artist.ellipsis').get_text().strip()
    album = tr.select_one('.albumtitle.ellipsis').get_text().strip()
    rank_list.append(rank), 
    title_list.append(title), 
    artist_list.append(artist), 
    album_list.append(album)

    

In [39]:
df = pd.DataFrame({
    '순위': rank_list, '제목' : title_list, '가수' : artist_list, '앨범' : album_list
})
df.head()

,순위,제목,가수,앨범
0,1,리무진 (Feat. MINO) (Prod. by GRAY),BE'O (비오),쇼미더머니 10 Episode 3
1,2,회전목마 (Feat. Zion.T & 원슈타인) (Prod. by Slom),sokodomo,쇼미더머니 10 Episode 2
2,3,만남은 쉽고 이별은 어려워 (Feat. Leellamarz) (Prod. by TOIL),베이식,쇼미더머니 10 Episode 3
3,4,불협화음 (Feat. AKMU) (Prod. by GRAY),Mudd the student,쇼미더머니 10 Semi Final
4,5,쉬어 (Feat. MINO) (Prod. by GRAY),아넌딜라이트 (Anandelight) & Unofficialboyy & BE'O (...,쇼미더머니 10 Episode 1


In [41]:
lines = []   
for tr in trs:
    rank = tr.select_one('.number').get_text().split('\n')[0]
    title = tr.select_one('.title.ellipsis').get_text().strip()
    artist = tr.select_one('.artist.ellipsis').get_text().strip()
    album = tr.select_one('.albumtitle.ellipsis').get_text().strip()
    lines.append([rank, title, artist, album])

df = pd.DataFrame(lines, columns = ['순위', '제목', '가수', '앨범'])
df.head()


,순위,제목,가수,앨범
0,1,리무진 (Feat. MINO) (Prod. by GRAY),BE'O (비오),쇼미더머니 10 Episode 3
1,2,회전목마 (Feat. Zion.T & 원슈타인) (Prod. by Slom),sokodomo,쇼미더머니 10 Episode 2
2,3,만남은 쉽고 이별은 어려워 (Feat. Leellamarz) (Prod. by TOIL),베이식,쇼미더머니 10 Episode 3
3,4,불협화음 (Feat. AKMU) (Prod. by GRAY),Mudd the student,쇼미더머니 10 Semi Final
4,5,쉬어 (Feat. MINO) (Prod. by GRAY),아넌딜라이트 (Anandelight) & Unofficialboyy & BE'O (...,쇼미더머니 10 Episode 1


In [42]:
lines = []   
for tr in trs:
    rank = tr.select_one('.number').get_text().split('\n')[0]
    title = tr.select_one('.title.ellipsis').get_text().strip()
    artist = tr.select_one('.artist.ellipsis').get_text().strip()
    album = tr.select_one('.albumtitle.ellipsis').get_text().strip()
    d={'순위' : rank , '제목' : title , '가수' : artist , '앨범' : album }
    lines.append(d)

df = pd.DataFrame(lines)
df.head()


,순위,제목,가수,앨범
0,1,리무진 (Feat. MINO) (Prod. by GRAY),BE'O (비오),쇼미더머니 10 Episode 3
1,2,회전목마 (Feat. Zion.T & 원슈타인) (Prod. by Slom),sokodomo,쇼미더머니 10 Episode 2
2,3,만남은 쉽고 이별은 어려워 (Feat. Leellamarz) (Prod. by TOIL),베이식,쇼미더머니 10 Episode 3
3,4,불협화음 (Feat. AKMU) (Prod. by GRAY),Mudd the student,쇼미더머니 10 Semi Final
4,5,쉬어 (Feat. MINO) (Prod. by GRAY),아넌딜라이트 (Anandelight) & Unofficialboyy & BE'O (...,쇼미더머니 10 Episode 1


### 5. 모든 페이지의 데이터 가져오기 

In [50]:
lines = []              # 전체 페이지 걸쳐서 line에 넣으므로 반복문 밖으로 빼주기

for page in range(1,5):
    url = f'https://www.genie.co.kr/chart/top200?ditc=D&ymd=20211206&hh=10&rtm=Y&pg={page}'
    req = requests.get(url, headers = header)
    html = req.text
    soup= BeautifulSoup(html, 'html.parser')
    tbody = soup.select_one('table.list-wrap').find('tbody')
    trs = tbody.select('tr')


    for tr in trs:                                                          # 반복문의 반복문!! 
        rank = tr.select_one('.number').get_text().split('\n')[0]
        title = tr.select_one('.title.ellipsis').get_text().strip()
        artist = tr.select_one('.artist.ellipsis').get_text().strip()
        album = tr.select_one('.albumtitle.ellipsis').get_text().strip()
        d={'순위' : rank , '제목' : title , '가수' : artist , '앨범' : album }
        lines.append(d)

df = pd.DataFrame(lines)
df.head()
    

,순위,제목,가수,앨범
0,1,리무진 (Feat. MINO) (Prod. by GRAY),BE'O (비오),쇼미더머니 10 Episode 3
1,2,회전목마 (Feat. Zion.T & 원슈타인) (Prod. by Slom),sokodomo,쇼미더머니 10 Episode 2
2,3,만남은 쉽고 이별은 어려워 (Feat. Leellamarz) (Prod. by TOIL),베이식,쇼미더머니 10 Episode 3
3,4,사랑은 늘 도망가,임영웅,신사와 아가씨 OST Part.2
4,5,불협화음 (Feat. AKMU) (Prod. by GRAY),Mudd the student,쇼미더머니 10 Semi Final


In [51]:
df.to_csv('지니차트_Top200(211206).csv', index = False)